# Setup

In [ ]:
# !pip install yfinance
# !pip install altair

In [ ]:
import pandas as pd
import altair as alt
import yfinance as yf

import matplotlib.pyplot as plt
%matplotlib inline

# Get Data

In [ ]:
# Get the data for the stock W
ticker = yf.download('W','2018-01-01','2022-12-31')

# Plot the close price of the W
ticker['Adj Close'].plot()
plt.show()

In [ ]:
ticker.sample(5)

# Data Prep

In [ ]:
# Source: https://infohub.corp.wayfair.com/pages/viewpage.action?pageId=31819745

df_trading_windows = pd.DataFrame([
    ['2019-11-04', '2019-11-29'], 
    ['2020-03-03', '2020-03-06'], 
    ['2020-05-07', '2020-05-29'],
    ['2020-08-07', '2020-08-31'], 
    ['2020-11-09', '2020-11-30'], 
    ['2021-02-26', '2021-03-05'], 
    ['2021-05-07', '2021-05-31'], 
    ['2021-08-06', '2021-08-31'],
    ['2021-11-05', '2021-11-30'],
    ['2022-02-25', '2022-03-07']
], columns=['dt_start', 'dt_end'], index=['2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2021Q1', '2021Q2', '2021Q3', '2021Q4'])
df_trading_windows

df_trading_windows['dt_start'] = pd.to_datetime(df_trading_windows['dt_start'])
df_trading_windows['dt_end'] = pd.to_datetime(df_trading_windows['dt_end'])


In [ ]:
df_trading_windows.dtypes

In [ ]:
# This is not elegant but optimizes for developer time

for i, period in df_trading_windows.iterrows():
    ticker.loc[period['dt_start']:period['dt_end'], 'period'] = '{}: {:%Y-%m-%d} - {:%Y-%m-%d}'.format(period.name, period['dt_start'], period['dt_end'])
    ticker.loc[period['dt_start']:period['dt_end'],  'day_td'] = ticker.loc[period['dt_start']:period['dt_end']].index - period['dt_start'] 
    ticker.loc[period['dt_start']:period['dt_end'],  'norm_close'] = ticker.loc[period['dt_start']:period['dt_end'], 'Close'] / ticker.loc[period['dt_start'], 'Open'] - 1.0
    ticker.loc[period['dt_start']:period['dt_end'],  'norm_high'] = ticker.loc[period['dt_start']:period['dt_end'], 'High'] / ticker.loc[period['dt_start'], 'Open'] - 1.0

ticker['day'] = ticker['day_td'].dt.days + 1

In [ ]:
ticker[~ticker['period'].isna()].tail()


# Charts

In [ ]:
df_chart = ticker.loc[~ticker['period'].isna(), ['Open', 'Close', 'norm_high', 'norm_close', 'period', 'day']]

## Daily Close vs. Window Open

In [ ]:

ch_close = alt.Chart(df_chart).mark_line().encode(
    x=alt.X("day:O", axis=alt.Axis(title='Calendar Days')),
    y=alt.Y("norm_close:Q", axis=alt.Axis(title='Daily Close vs. Day 1 Open', format='%')),
    color=alt.Color("period:N", scale=alt.Scale(scheme='tableau20')),
    strokeWidth=alt.condition(
        "datum.period == '2021Q4: 2022-02-25 - 2022-03-07'",
        alt.value(5), alt.value(2))
).properties(title="W") 

ch_close

In [ ]:

ch_close = alt.Chart(df_chart).mark_line().encode(
    x=alt.X("day:O", axis=alt.Axis(title='Calendar Days')),
    y=alt.Y("norm_close:Q", axis=alt.Axis(title='Daily Close vs. Day 1 Open', format='%')),
    row=alt.Row("period:N")
    #, color=alt.Color("norm_close:Q")
).properties(title="W", height=50, width=400)

ch_close

## Daily High vs. Window Open

In [ ]:

ch_high = alt.Chart(df_chart).mark_line().encode(
    x=alt.X("day:O", axis=alt.Axis(title='Calendar Days')),
    y=alt.Y("norm_high:Q", axis=alt.Axis(title='Daily High vs. Day 1 Open', format='%')),
    color=alt.Color("period:N", scale=alt.Scale(scheme='tableau20')),
    strokeWidth=alt.condition(
        "datum.period == '2021Q3: 2021-11-05 - 2021-11-30'",
        alt.value(5), alt.value(2))
)

ch_high